<h1>Exercise set 6: Fast Fourier Transform (part 2)</h1>

<p><span style="color: #0000ff;">Th next cell contains a functions: <span style="font-family: comic sans ms,sans-serif;">root_of_unity</span> and <span style="font-family: courier new,courier;">split_list</span> from exercise sets 3-5. It will get executed automatically (the command <span style="font-family: courier new,courier;">%auto</span> at the top serves this purpose)</span><strong><br /></strong></p>

In [12]:

def root_of_unity( n, K = CC ) :
    """
    This function computes the primitive root of unity of degree `n` in `\\mathbb{C}`
    """
    return K( cos(2*pi/n) + i*sin(2*pi/n) )

def split_list( L ) :
    """
    Given a list `L` this function constructs lists `E` with elements of `L` having even indices \
    and `O` with elements of `L` having odd indices.
    """
    n = len(L)
    E = [ L[j] for j in [0,2,..,n-1] ]
    O = [ L[j] for j in [1,3,..,n-1] ]
    return E,O

<p><strong>Exercise:</strong> Write a function that computes DFT using Cooley-Tukey algorithm.</p>

In [19]:
def fft( L, omega = 0 ) :
    """
    This function computes FFT of a list `L`, using Cooley-Tukey algorithm. \
    The length of the list `L` must be a power of 2.
    
    INPUT:
        
        - `L` list
        
        - `omega` - primitive root of unity of degree `|L|`.
        If `omega` is omited, it will be computed.
    """
    # get the length of L
    n = len(L)
    # stop the recursion if the list consists of a single element
    if n == 1 :
        return L
    assert NN(n).is_power_of(2), "The length of L must be a power of 2"
    # compute omega if it was not provided
    if omega == 0:
        omega = root_of_unity(n)

    # split L into its even and odd parts
    e, o = split_list(L)

    # compute FFTs of both parts 
    E = fft(e, omega^2)
    O = fft(o, omega^2)

    # recombine the partial results
    L0 = [ E[j] + omega^(-j)*O[j] for j in [0..n//2-1] ] 
    L1 = [ E[j] - omega^(-j)*O[j] for j in [0..n//2-1] ] 

    # return the final result
    return L0 + L1

In [26]:
# verification
fft( [1..4] )

[10.0000000000000,
 -2.00000000000000 + 2.00000000000000*I,
 -2.00000000000000,
 -2.00000000000000 - 2.00000000000000*I]

In [27]:
# verification
fft( [1..8], root_of_unity(8, SR) )

[36,
 4*I*sqrt(2) + 4*I - 4,
 4*I - 4,
 4*I*sqrt(2) - 4*I - 4,
 -4,
 -4*I*sqrt(2) + 4*I - 4,
 -4*I - 4,
 -4*I*sqrt(2) - 4*I - 4]

<p><strong>Exercise:</strong> Compute FFT's of sequences $(1,\dotsc, 2^k)$, for $k\leq 10$ using respectively: directly the definition, theorem about values of a polynomial, Cooley-Tukey algorithm. Compare the execution times of each method.</p>

In [3]:
# construct the sequences
seqs = [ [1..2^k] for k in [1..10] ];

In [29]:
%%timeit
# directly by definition

for l in seqs :
    omega = root_of_unity( len(l) )
    L = [ sum( l[k]*omega^(-k*n) for k in range(len(l)) ) for n in range(len(l)) ]

CPU time: 11.01 s,  Wall time: 11.01 s

In [31]:
%%timeit
# using the theorem
P.<x> = ZZ[]

for l in seqs :
    f = P(l)
    omega = root_of_unity( f.degree() )
    F = [ f(omega^(-j)) for j in [0..f.degree()] ]

CPU time: 1.40 s,  Wall time: 1.40 s

In [32]:
%%timeit
# using Cooley-Tukey

for l in seqs :
    F = fft(l)

CPU time: 0.23 s,  Wall time: 0.23 s

<p><strong>Exercise:</strong> Write a function that computes the next power of 2 greater or equal than a given number $n$. In other words it computes $2^N$ such that $2^{N-1} < n \leq 2^N$.</p>

In [46]:
def next_pow_2( n ) :
    """
    This function computes `2^N` s.t. `2^{N-1} < n \\leq 2^N`
    """
    assert n > 0, "n must be positive!"
    N = ZZ(ceil(log(n, 2)))
    return 2^N

In [49]:
# verification
for j in [1..16] :
    print j, '\t', next_pow_2(j)

1 	1
2 	2
3 	4
4 	4
5 	8
6 	8
7 	8
8 	8
9 	16
10 	16
11 	16
12 	16
13 	16
14 	16
15 	16
16 	16

<p><strong>Exercise:</strong> Multiply two random polynomials with integer coefficients using fast Fourier transform.</p>
<p>1. Construct two random elements of the ring $\mathbb{Z}[x]$.</p>

In [33]:
P.<x> = ZZ[]
f = P.random_element(7); view("f = " + latex(f))
g = P.random_element(5); view("g = " + latex(g))

<p>2. Find the length of the sequence for FFT.</p>

In [45]:
# find the length of the resulting sequence
N = 2*max( next_pow_2(f.degree()), next_pow_2(g.degree()) )
print N

16

<p>3. Convert both polynomials into lists and append the list with needed numbers of zeros.</p>

In [47]:
# convert f, g into lists of length N
f_ = f.list() + [0]*(N - f.degree() - 1); print f_
g_ = g.list() + [0]*(N - g.degree() - 1); print g_

[9, 0, -4, 3, -1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[-1, -1, -1, -1, 1, -3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

<p>4. Compute discrete Fourier transforms of both lists. Do it in two ways: first use a complex root of unity, secondly use a cyclotomic field (command <tt>CyclotomicField</tt>). </p>

In [48]:
# compute FFT of both lists
omega = root_of_unity(N)
#K.<omega> = CyclotomicField(N)
F = fft(f_, omega); print F
G = fft(g_, omega); print G

[10.0000000000000, 5.30595342628615 - 0.956881218850594*I, 7.87867965644036 + 4.29289321881345*I, 10.3050914085451 + 1.72817454050872*I, 11.0000000000000 + 3.00000000000000*I, 14.7659764033204 - 0.514466146610572*I, 12.1213203435596 - 5.70710678118655*I, 5.62297876184836 - 7.19952190596988*I, 0.000000000000000, 5.62297876184837 + 7.19952190596988*I, 12.1213203435596 + 5.70710678118655*I, 14.7659764033204 + 0.514466146610567*I, 11.0000000000000 - 3.00000000000000*I, 10.3050914085451 - 1.72817454050871*I, 7.87867965644036 - 4.29289321881345*I, 5.30595342628616 + 0.956881218850597*I]
[-6.00000000000000, -1.86561944896765 + 3.78530834359678*I, 0.121320343559643 + 0.292893218813452*I, -2.52333571620111 + 1.10025258423748*I, 1.00000000000000 + 3.00000000000000*I, 1.93754927857421 - 2.31396097813562*I, -4.12132034355964 - 1.70710678118655*I, -1.54859411340544 + 4.37109478122369*I, 4.00000000000000, -1.54859411340544 - 4.37109478122369*I, -4.12132034355964 + 1.70710678118655*I, 1.9375492785742

<p>5. Multiply the resulting sequences coordinate-wisely.</p>

In [52]:
H = [ F[j]*G[j] for j in [0..N-1] ]; print H

[-60.0000000000000, -6.27679944584980 + 21.8698459874965*I, -0.301515190164998 + 2.82842712474619*I, -27.9046337141072 + 6.97743891116004*I, 2.00000000000001 + 36.0000000000000*I, 27.4193523398686 - 35.1646967125710*I, -59.6984848098350 + 2.82842712474622*I, 22.7620808200884 + 35.7277103637655*I, 0.000000000000000, 22.7620808200884 - 35.7277103637655*I, -59.6984848098350 - 2.82842712474617*I, 27.4193523398686 + 35.1646967125709*I, 1.99999999999998 - 36.0000000000000*I, -27.9046337141072 - 6.97743891116002*I, -0.301515190165008 - 2.82842712474620*I, -6.27679944584982 - 21.8698459874965*I]

<p>6. Compute the inverse transform.</p>

In [53]:
# the inverse transform = "forward" transform with omega replaced by 1/omega (remember about 1/N scaling)
h_ = [ hi/N for hi in fft(H, omega^(-1)) ]; print h_

[-9.00000000000000 - 5.49560397189452e-15*I, -9.00000000000000 + 4.99600361081320e-16*I, -5.00000000000000 - 2.88657986402541e-15*I, -8.00000000000000 + 1.27675647831893e-15*I, 11.0000000000000 - 4.38538094726937e-15*I, -26.0000000000000 + 4.99600361081320e-16*I, -8.00000000000000 - 4.44089209850063e-16*I, 13.0000000000000 + 8.32667268468868e-16*I, -13.0000000000000 + 1.13797860024079e-14*I, 2.00000000000000 + 4.99600361081320e-16*I, -3.00000000000000 + 1.99840144432528e-15*I, -2.00000000000000 - 4.99600361081320e-16*I, -3.00000000000000 + 2.77555756156292e-16*I, -1.77635683940025e-15 - 1.27675647831893e-15*I, 3.99680288865056e-15 - 4.44089209850063e-16*I, 8.88178419700125e-16 - 1.83186799063151e-15*I]

<p>6a. Convert the results back into integers.</p>

In [55]:
h_ = [ round(real_part(hi)) for hi in h_ ]; print h_

[-9, -9, -5, -8, 11, -26, -8, 13, -13, 2, -3, -2, -3, 0, 0, 0]

<p>7. Interpret the resulting sequence as a polynomial.</p>

In [56]:
# cast onto a polynomial type
h = P(h_); view("f\\cdot g = " + latex(h))

In [57]:
# verification
h == f*g

True